# Basket option implementation with Bachelier model CV

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import numpy as np
import pyfeng as pf

#import option_models as opt
from option_models import basket

In [27]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [28]:
cor_m

array([[1. , 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5],
       [0.5, 0.5, 0.5, 1. ]])

In [29]:
print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket)

[1 0 0 0]
26.570984651213593


In [30]:
# Compare the price to normal model formula

norm1 = pf.Norm(sigma=40)
price_norm = norm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_norm)

26.570984651213593 26.570845957870503


In [31]:
# A trivial test case 2
# all assets almost perfectly correlated:
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 5
strike = 120

print( cor_m )

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_norm)

[[1.     0.9999 0.9999 0.9999]
 [0.9999 1.     0.9999 0.9999]
 [0.9999 0.9999 1.     0.9999]
 [0.9999 0.9999 0.9999 1.    ]]
26.57211110181949 26.570845957870503


In [32]:
# A full test set for basket option with exact price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 100
price_exact = 28.0073695

In [33]:
weights, cor_m

(array([0.25, 0.25, 0.25, 0.25]), array([[1. , 0.5, 0.5, 0.5],
        [0.5, 1. , 0.5, 0.5],
        [0.5, 0.5, 1. , 0.5],
        [0.5, 0.5, 0.5, 1. ]]))

In [34]:
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_exact)

28.22782882893394 28.0073695


# [To Do] Basket option implementation based on BSM model
## Write the similar test for BSM

In [35]:
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)
print(price_basket)

27.967309091912327


In [36]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)

[1 0 0 0]


In [37]:
bsm1 = pf.Bsm(sigma=vol[0])
price_bsm = bsm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_bsm)

28.514393651998596 28.713486748445934


# Spread option implementation based on normal model

In [38]:
# A full test set for spread option

spot = np.array([100, 96])
vol = np.array([0.2, 0.1])
weights = np.array([1, -1])
divr = np.array([1, 1])*0.05
intr = 0.1
cor = 0.5
cor_m = np.array([[1, cor], [cor, 1]])
texp = 1
strike = 0
price_exact = 8.5132252

In [39]:
# MC price based on normal model
# make sure that the prices are similar

np.random.seed(123456)
price_spread = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=False)
print(price_spread, price_exact)

8.317680907159142 8.5132252


# Spread option implementation based on BSM model

In [40]:
# Once the implementation is finished the BSM model price should also work
price_spread = basket.basket_price_mc(
    strike, spot, vol, weights, texp, cor_m, intr=intr, divr=divr, bsm=True)
price_spread

8.530935560352313

In [42]:
# You also test Kirk's approximation
kirk = pf.BsmSpreadKirk(vol, cor=cor, divr=divr, intr=intr)
price_kirk = kirk.price(strike, spot, texp)
print(price_kirk, price_spread)

8.513225229545505 8.530935560352313


# [To Do] Complete the implementation of basket_price_norm_analytic
# Compare the MC stdev of BSM basket prices from with and without CV

In [43]:
# The basket option example from above
spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 1, 1, 1])/4
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [46]:
### Make sure that the analytic normal price is correctly implemented
basket.basket_price_norm_analytic(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr)

19.330395569726363

In [58]:
# Run below about 100 times and get the mean and stdev

### Returns 2 prices, without CV and with CV 
result=np.zeros(shape=(100,2))
for i in range(100):
    price_basket = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)
    result[i,:]=price_basket
mean_without_cv=np.mean(result[:,0])
mean_with_cv=np.mean(result[:,1])
stdev_without_cv=np.std(result[:,0])
stdev_with_cv=np.std(result[:,1])
print(mean_without_cv, mean_with_cv)
print(stdev_without_cv, stdev_with_cv)

21.846262723367822 21.80459751926518
0.5839033218407825 0.34861268901688763


In [55]:
print(result)

[[21.68122514 21.64000961]
 [21.61407341 21.74872574]
 [21.88120818 21.78417659]
 [20.96696968 21.50458401]
 [21.75711901 21.67280828]
 [22.50046706 22.39371109]
 [20.88812737 21.70401853]
 [22.12811987 22.00479216]
 [22.87403192 22.36631445]
 [21.58226416 21.43355241]
 [21.73036959 21.77636647]
 [22.1067677  22.22910494]
 [21.26603705 21.63860384]
 [22.0518206  21.75573323]
 [22.84147046 22.18168071]
 [22.31486655 22.13020785]
 [20.88784223 21.28406036]
 [21.10712089 21.48916264]
 [21.09699775 21.47659112]
 [22.78116977 22.2005955 ]
 [21.47275083 21.75962814]
 [23.06187918 22.45955597]
 [22.33558083 22.15049136]
 [22.88782296 22.51205073]
 [20.95405175 21.29627157]
 [21.93447499 21.76389661]
 [21.79012592 21.71326792]
 [21.30147762 21.56528061]
 [22.56765452 22.1165392 ]
 [21.29197749 21.30112594]
 [22.99448117 22.3185075 ]
 [22.02622992 21.73642118]
 [21.32903359 21.379808  ]
 [20.96558383 21.25877991]
 [21.81753188 21.96521053]
 [22.82851766 22.23026551]
 [22.38951434 22.05562868]
 